In [1]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 11.5 MB/s eta 0:00:00


In [4]:
from __future__ import print_function, division
import numpy as np
import pandas as pd

import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support, f1_score, classification_report, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold

import optuna
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import xgboost as xgb
torch.manual_seed(15)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv("/content/drive/My Drive/BT4012 Team 8/Ethereum/feature_data.csv")

In [ ]:
df

,node,isp,closeness_centrality,betweenness_centrality,eigenvector_centrality,active_days,eccentricity,pagerank_std_last_month,in_out_degree_ratio,weightsin_weightsout_ratio,numin_numout_ratio,centrality_sum,rolling_average_pagerank,cumulative_interaction_count,pagerank_change,pagerank_closeness_interaction
0,0x1f1e784a61a8ca0a90250bcd2170696655b28a21,0,0.132361,2.290699e-04,2.496896e-03,107.0,7.0,0.000000e+00,0.132389,1.000000,1.0,1.350873e-01,0.000000e+00,1327.0,0.000000e+00,8.164333e-08
1,0x1266f8b9e4dffc9e2f719bf51713f7e714516861,0,0.109312,1.747367e-07,1.795760e-05,1.0,8.0,0.000000e+00,1.000000,1.000000,1.0,1.093298e-01,0.000000e+00,1328.0,-4.834848e-07,1.457527e-08
2,0xbbfaf27674c2eb5d13edc58a40081248d13dcfeb,1,0.117714,7.214720e-05,7.059330e-05,0.0,7.0,0.000000e+00,1.000000,1.000000,0.0,1.178565e-01,5.317118e-07,0.0,7.116401e-07,9.946543e-08
3,0xb50d0c4cb2c29cc232c96a59e9c65eb82914ec75,0,0.110925,1.036793e-04,2.117960e-03,89.0,7.0,0.000000e+00,0.379518,0.434854,1.0,1.131464e-01,6.202811e-07,1445.0,7.495447e-07,9.791388e-08
4,0x563b377a956c80d77a7c613a9343699ad6123911,0,0.110763,5.919153e-03,6.486499e-03,346.0,7.0,0.000000e+00,0.010035,0.000029,1.0,1.231683e-01,4.186281e-07,9354.0,-7.983363e-07,1.538709e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29456,0xdcf880ba3487a4e95259d79994226b1e4f8d9458,0,0.000000,0.000000e+00,1.285257e-05,1.0,8.0,1.037354e-07,0.666667,1.000000,1.0,1.285257e-05,3.237338e-07,5333250.0,5.717189e-07,0.000000e+00
29457,0x2ff58a66bf3073e4c6016e1a5d8d8eb6483f5723,0,0.120039,0.000000e+00,4.736295e-04,1.0,8.0,0.000000e+00,1.000000,0.288970,0.5,1.205131e-01,1.331627e-07,5337613.0,5.578834e-12,1.598523e-08
29458,0x0d8415be40925d965ade2cd62be5680389d640e4,0,0.110962,6.770483e-06,3.166462e-05,1.0,8.0,2.400348e-08,0.500000,0.504685,1.0,1.110003e-01,1.985535e-07,5338232.0,-8.968140e-08,1.539773e-08
29459,0xe4edc0ee94ba28ac89d7304c72fa95081a3bef72,0,0.086852,2.634004e-07,1.122154e-04,2.0,8.0,3.365868e-08,0.500000,0.981012,1.0,8.696465e-02,1.845676e-07,5338238.0,-1.316392e-07,1.254592e-08


**Train and test data**

In [6]:
X = df.drop(['isp', 'node'], axis=1)
y = df['isp']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=15,shuffle=False)

Fine-tuning

In [7]:
rsn=15
kf = StratifiedKFold(n_splits=3)

def objective_xgb(trial):
    n_estimators = trial.suggest_int('n_estimators', 20, 320)
    max_depth = trial.suggest_int('max_depth', 1, 20)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 20)
    gamma = trial.suggest_float('gamma', 0, 1)
    subsample = trial.suggest_float('subsample', 0.1, 1.0)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.1, 1.0)

    params = {
        'max_depth': max_depth,
        'learning_rate': learning_rate,
        'min_child_weight': min_child_weight,
        'gamma': gamma,
        'subsample': subsample,
        'colsample_bytree': colsample_bytree
    }
    model = xgb.XGBClassifier(n_estimators=n_estimators, **params)
    f1_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='f1_micro')
    return f1_scores.mean()

def print_trial_callback(study, trial):
    print(f"Trial {trial.number}: {trial.value}")
    print(f"Params: {trial.params}")

In [8]:
#XGB
study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(objective_xgb, n_trials=30, callbacks=[print_trial_callback])

xgb_best_params = study_xgb.best_trial.params
xgbc = xgb.XGBClassifier(**xgb_best_params, random_state=rsn).fit(X_train, y_train)
xgb_preds = xgbc.predict(X_test)
xgb_prec, xgb_rec, xgb_f1, xgb_num = precision_recall_fscore_support(y_test, xgb_preds, average=None)

[I 2023-11-21 17:33:14,821] A new study created in memory with name: no-name-738f7017-9c43-4e8f-a36c-434d36b212b3
[I 2023-11-21 17:33:16,854] Trial 0 finished with value: 0.949713897779071 and parameters: {'n_estimators': 184, 'max_depth': 4, 'learning_rate': 0.0023836541902716953, 'min_child_weight': 15, 'gamma': 0.755033897109112, 'subsample': 0.332586617966419, 'colsample_bytree': 0.6910455281527315}. Best is trial 0 with value: 0.949713897779071.


Trial 0: 0.949713897779071
Params: {'n_estimators': 184, 'max_depth': 4, 'learning_rate': 0.0023836541902716953, 'min_child_weight': 15, 'gamma': 0.755033897109112, 'subsample': 0.332586617966419, 'colsample_bytree': 0.6910455281527315}


[I 2023-11-21 17:33:18,879] Trial 1 finished with value: 0.9646009116477549 and parameters: {'n_estimators': 214, 'max_depth': 4, 'learning_rate': 0.00683663205335453, 'min_child_weight': 12, 'gamma': 0.9660530033642786, 'subsample': 0.39188457530370824, 'colsample_bytree': 0.7439233745824764}. Best is trial 1 with value: 0.9646009116477549.


Trial 1: 0.9646009116477549
Params: {'n_estimators': 214, 'max_depth': 4, 'learning_rate': 0.00683663205335453, 'min_child_weight': 12, 'gamma': 0.9660530033642786, 'subsample': 0.39188457530370824, 'colsample_bytree': 0.7439233745824764}


[I 2023-11-21 17:33:20,226] Trial 2 finished with value: 0.9129570361749589 and parameters: {'n_estimators': 137, 'max_depth': 7, 'learning_rate': 0.03232854295150346, 'min_child_weight': 2, 'gamma': 0.01257761117592271, 'subsample': 0.4857033386263895, 'colsample_bytree': 0.6930236544309472}. Best is trial 1 with value: 0.9646009116477549.


...

Trial 27: 0.949713897779071
Params: {'n_estimators': 199, 'max_depth': 10, 'learning_rate': 0.003948985311513213, 'min_child_weight': 3, 'gamma': 0.6446336932557651, 'subsample': 0.9987208173832598, 'colsample_bytree': 0.21315285087333607}


[I 2023-11-21 17:34:28,657] Trial 28 finished with value: 0.9609640190088257 and parameters: {'n_estimators': 162, 'max_depth': 6, 'learning_rate': 0.010891017469270835, 'min_child_weight': 9, 'gamma': 0.9211713297592048, 'subsample': 0.7555176312704581, 'colsample_bytree': 0.3224020046707867}. Best is trial 11 with value: 0.9662496363107361.


Trial 28: 0.9609640190088257
Params: {'n_estimators': 162, 'max_depth': 6, 'learning_rate': 0.010891017469270835, 'min_child_weight': 9, 'gamma': 0.9211713297592048, 'subsample': 0.7555176312704581, 'colsample_bytree': 0.3224020046707867}


[I 2023-11-21 17:34:30,397] Trial 29 finished with value: 0.9537387256328193 and parameters: {'n_estimators': 277, 'max_depth': 5, 'learning_rate': 0.0023481685727903294, 'min_child_weight': 13, 'gamma': 0.7772861293397555, 'subsample': 0.8214894417972907, 'colsample_bytree': 0.6179042579929437}. Best is trial 11 with value: 0.9662496363107361.


Trial 29: 0.9537387256328193
Params: {'n_estimators': 277, 'max_depth': 5, 'learning_rate': 0.0023481685727903294, 'min_child_weight': 13, 'gamma': 0.7772861293397555, 'subsample': 0.8214894417972907, 'colsample_bytree': 0.6179042579929437}


In [9]:
# Evaluation
print("\n\nXGB Classifier Model")
print("Precision: %.3f \nRecall: %.3f \nF1 Score: %.3f" % (xgb_prec[1], xgb_rec[1], xgb_f1[1]))
micro_f1 = f1_score(y_test, xgb_preds, average='micro')
print("Micro-Average F1 Score:", micro_f1)
classification = classification_report(y_test, xgb_preds)
print("Classification Report: \n", classification)

# roc-auc
y_pred_prob = xgbc.predict_proba(X_test)[:, 1]
auc_test = roc_auc_score(y_test, y_pred_prob)
print(f"ROC-AUC score is {auc_test}")



XGB Classifier Model
Precision: 0.944 
Recall: 0.266 
F1 Score: 0.415
Micro-Average F1 Score: 0.9891390428781537
Classification Report: 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      8711
           1       0.94      0.27      0.41       128

    accuracy                           0.99      8839
   macro avg       0.97      0.63      0.70      8839
weighted avg       0.99      0.99      0.99      8839

ROC-AUC score is 0.9538900169326139


In [ ]:
# Best params
xgb_best_params

{'n_estimators': 56,
 'max_depth': 17,
 'learning_rate': 0.03592129078025108,
 'min_child_weight': 1,
 'gamma': 0.719694736628453,
 'subsample': 0.5787114492912084,
 'colsample_bytree': 0.5386269645943523}

## Apply Undersampling

In [10]:
# Apply Undersampling
from imblearn.under_sampling import NearMiss
undersample = NearMiss(version=3, n_neighbors=10)
X_under, y_under = undersample.fit_resample(X_train, y_train)

In [11]:
#XGB
study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(objective_xgb, n_trials=30, callbacks=[print_trial_callback])

xgb_best_params = study_xgb.best_trial.params
xgbc = xgb.XGBClassifier(**xgb_best_params, random_state=rsn).fit(X_under, y_under)
xgb_preds = xgbc.predict(X_test)
xgb_prec, xgb_rec, xgb_f1, xgb_num = precision_recall_fscore_support(y_test, xgb_preds, average=None)

[I 2023-11-21 17:35:03,418] A new study created in memory with name: no-name-cf82023a-70cd-4282-b67a-81bdaa5cad46
[I 2023-11-21 17:35:04,940] Trial 0 finished with value: 0.9116962467267967 and parameters: {'n_estimators': 141, 'max_depth': 19, 'learning_rate': 0.09293288497890698, 'min_child_weight': 14, 'gamma': 0.2152912984837415, 'subsample': 0.7196677835825392, 'colsample_bytree': 0.8836000740930237}. Best is trial 0 with value: 0.9116962467267967.


Trial 0: 0.9116962467267967
Params: {'n_estimators': 141, 'max_depth': 19, 'learning_rate': 0.09293288497890698, 'min_child_weight': 14, 'gamma': 0.2152912984837415, 'subsample': 0.7196677835825392, 'colsample_bytree': 0.8836000740930237}


[I 2023-11-21 17:35:05,938] Trial 1 finished with value: 0.949713897779071 and parameters: {'n_estimators': 69, 'max_depth': 9, 'learning_rate': 0.0011024580681246658, 'min_child_weight': 4, 'gamma': 0.7892257717984397, 'subsample': 0.6134862527383005, 'colsample_bytree': 0.2675050286461773}. Best is trial 1 with value: 0.949713897779071.


Trial 1: 0.949713897779071
Params: {'n_estimators': 69, 'max_depth': 9, 'learning_rate': 0.0011024580681246658, 'min_child_weight': 4, 'gamma': 0.7892257717984397, 'subsample': 0.6134862527383005, 'colsample_bytree': 0.2675050286461773}


[I 2023-11-21 17:35:08,292] Trial 2 finished with value: 0.949713897779071 and parameters: {'n_estimators': 147, 'max_depth': 9, 'learning_rate': 0.0010665475307863877, 'min_child_weight': 9, 'gamma': 0.21525752759411954, 'subsample': 0.9467894313830515, 'colsample_bytree': 0.7824741921633988}. Best is trial 1 with value: 0.949713897779071.


...

Trial 27: 0.9528658713994762
Params: {'n_estimators': 193, 'max_depth': 4, 'learning_rate': 0.010378512178295324, 'min_child_weight': 17, 'gamma': 0.5873501037466271, 'subsample': 0.7347943863610078, 'colsample_bytree': 0.8186289752863031}


[I 2023-11-21 17:36:05,236] Trial 28 finished with value: 0.9616913975366114 and parameters: {'n_estimators': 246, 'max_depth': 8, 'learning_rate': 0.004746864713508504, 'min_child_weight': 20, 'gamma': 0.5080569072288894, 'subsample': 0.7840254250464109, 'colsample_bytree': 0.8310015263541279}. Best is trial 23 with value: 0.9649403549607215.


Trial 28: 0.9616913975366114
Params: {'n_estimators': 246, 'max_depth': 8, 'learning_rate': 0.004746864713508504, 'min_child_weight': 20, 'gamma': 0.5080569072288894, 'subsample': 0.7840254250464109, 'colsample_bytree': 0.8310015263541279}


[I 2023-11-21 17:36:06,068] Trial 29 finished with value: 0.939676074095626 and parameters: {'n_estimators': 149, 'max_depth': 3, 'learning_rate': 0.03386644120433061, 'min_child_weight': 13, 'gamma': 0.3940709175173903, 'subsample': 0.7069977770968408, 'colsample_bytree': 0.8691496566409318}. Best is trial 23 with value: 0.9649403549607215.


Trial 29: 0.939676074095626
Params: {'n_estimators': 149, 'max_depth': 3, 'learning_rate': 0.03386644120433061, 'min_child_weight': 13, 'gamma': 0.3940709175173903, 'subsample': 0.7069977770968408, 'colsample_bytree': 0.8691496566409318}


In [12]:
# Evaluation
print("\n\nXGB Classifier Model")
print("Precision: %.3f \nRecall: %.3f \nF1 Score: %.3f" % (xgb_prec[1], xgb_rec[1], xgb_f1[1]))
micro_f1 = f1_score(y_test, xgb_preds, average='micro')
print("Micro-Average F1 Score:", micro_f1)
classification = classification_report(y_test, xgb_preds)
print("Classification Report: \n", classification)

# roc-auc
y_pred_prob = xgbc.predict_proba(X_test)[:, 1]
auc_test = roc_auc_score(y_test, y_pred_prob)
print(f"ROC-AUC score is {auc_test}")



XGB Classifier Model
Precision: 0.188 
Recall: 0.562 
F1 Score: 0.281
Micro-Average F1 Score: 0.9583663310329222
Classification Report: 
               precision    recall  f1-score   support

           0       0.99      0.96      0.98      8711
           1       0.19      0.56      0.28       128

    accuracy                           0.96      8839
   macro avg       0.59      0.76      0.63      8839
weighted avg       0.98      0.96      0.97      8839

ROC-AUC score is 0.931328295402365


## Apply Oversampling

In [13]:
from imblearn.over_sampling import RandomOverSampler

oversample = RandomOverSampler(sampling_strategy='minority')
X_over, y_over = oversample.fit_resample(X_train, y_train)
#XGB
study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(objective_xgb, n_trials=30, callbacks=[print_trial_callback])

xgb_best_params = study_xgb.best_trial.params
xgbc = xgb.XGBClassifier(**xgb_best_params, random_state=rsn).fit(X_over, y_over)
xgb_preds = xgbc.predict(X_test)
xgb_prec, xgb_rec, xgb_f1, xgb_num = precision_recall_fscore_support(y_test, xgb_preds, average=None)

[I 2023-11-21 17:36:06,474] A new study created in memory with name: no-name-381ec918-defd-4deb-87fa-05777bbc9b6e
[I 2023-11-21 17:36:09,124] Trial 0 finished with value: 0.949713897779071 and parameters: {'n_estimators': 151, 'max_depth': 11, 'learning_rate': 0.00011124749478807871, 'min_child_weight': 1, 'gamma': 0.3744079385317992, 'subsample': 0.847674276710867, 'colsample_bytree': 0.7072505039835323}. Best is trial 0 with value: 0.949713897779071.


Trial 0: 0.949713897779071
Params: {'n_estimators': 151, 'max_depth': 11, 'learning_rate': 0.00011124749478807871, 'min_child_weight': 1, 'gamma': 0.3744079385317992, 'subsample': 0.847674276710867, 'colsample_bytree': 0.7072505039835323}


[I 2023-11-21 17:36:10,733] Trial 1 finished with value: 0.949713897779071 and parameters: {'n_estimators': 290, 'max_depth': 16, 'learning_rate': 0.001495401058410671, 'min_child_weight': 17, 'gamma': 0.5291048357432563, 'subsample': 0.37581340802139296, 'colsample_bytree': 0.22089491879040002}. Best is trial 0 with value: 0.949713897779071.


Trial 1: 0.949713897779071
Params: {'n_estimators': 290, 'max_depth': 16, 'learning_rate': 0.001495401058410671, 'min_child_weight': 17, 'gamma': 0.5291048357432563, 'subsample': 0.37581340802139296, 'colsample_bytree': 0.22089491879040002}


[I 2023-11-21 17:36:12,957] Trial 2 finished with value: 0.949713897779071 and parameters: {'n_estimators': 190, 'max_depth': 8, 'learning_rate': 0.00019802463006663682, 'min_child_weight': 4, 'gamma': 0.746965447962918, 'subsample': 0.9201477982381807, 'colsample_bytree': 0.8465163511570648}. Best is trial 0 with value: 0.949713897779071.


...

Trial 27: 0.9102899815730773
Params: {'n_estimators': 182, 'max_depth': 15, 'learning_rate': 0.09527960760224177, 'min_child_weight': 6, 'gamma': 0.3499971868756794, 'subsample': 0.8998762698182956, 'colsample_bytree': 0.47733529265167424}


[I 2023-11-21 17:36:57,405] Trial 28 finished with value: 0.9668315391329648 and parameters: {'n_estimators': 320, 'max_depth': 19, 'learning_rate': 0.005487509274897951, 'min_child_weight': 10, 'gamma': 0.47259191487101354, 'subsample': 0.8438527214267867, 'colsample_bytree': 0.5794464551095239}. Best is trial 25 with value: 0.9679953447774222.


Trial 28: 0.9668315391329648
Params: {'n_estimators': 320, 'max_depth': 19, 'learning_rate': 0.005487509274897951, 'min_child_weight': 10, 'gamma': 0.47259191487101354, 'subsample': 0.8438527214267867, 'colsample_bytree': 0.5794464551095239}


[I 2023-11-21 17:36:59,706] Trial 29 finished with value: 0.9655222577829502 and parameters: {'n_estimators': 267, 'max_depth': 19, 'learning_rate': 0.004897981643845151, 'min_child_weight': 10, 'gamma': 0.3811060530774715, 'subsample': 0.7756799197929898, 'colsample_bytree': 0.6382791143861735}. Best is trial 25 with value: 0.9679953447774222.


Trial 29: 0.9655222577829502
Params: {'n_estimators': 267, 'max_depth': 19, 'learning_rate': 0.004897981643845151, 'min_child_weight': 10, 'gamma': 0.3811060530774715, 'subsample': 0.7756799197929898, 'colsample_bytree': 0.6382791143861735}


In [14]:
# Evaluation
print("\n\nXGB Classifier Model")
print("Precision: %.3f \nRecall: %.3f \nF1 Score: %.3f" % (xgb_prec[1], xgb_rec[1], xgb_f1[1]))
micro_f1 = f1_score(y_test, xgb_preds, average='micro')
print("Micro-Average F1 Score:", micro_f1)
classification = classification_report(y_test, xgb_preds)
print("Classification Report: \n", classification)

# roc-auc
y_pred_prob = xgbc.predict_proba(X_test)[:, 1]
auc_test = roc_auc_score(y_test, y_pred_prob)
print(f"ROC-AUC score is {auc_test}")



XGB Classifier Model
Precision: 0.479 
Recall: 0.625 
F1 Score: 0.542
Micro-Average F1 Score: 0.9847267790474036
Classification Report: 
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      8711
           1       0.48      0.62      0.54       128

    accuracy                           0.98      8839
   macro avg       0.74      0.81      0.77      8839
weighted avg       0.99      0.98      0.99      8839

ROC-AUC score is 0.9665311818390541
